In [99]:
from bioexplorer import BioExplorer, Protein, ParametricMembrane, Cell, Volume, Vector2, Vector3, Quaternion
from circuitexplorer import CircuitExplorer

url = 'localhost:5000'
be = BioExplorer(url)
ce = CircuitExplorer(be.core_api())
be.reset()

{'contents': 'Removed 6 models', 'status': True}

In [100]:
simulation_guid = 4

generate_snapshot = False
generate_movie = True

be.set_general_settings(model_visibility_on_creation=False)

representation=be.REPRESENTATION_ATOMS_AND_STICKS

scene_size = Vector3(200.0, 500.0, 200.0)
membrane_size = Vector2(scene_size.x, scene_size.z)
membrane_height = scene_size.y / 50.0
membrane_nb_lipids = int(membrane_size.x * membrane_size.y / 4.3) 

In [101]:
resource_folder = '../../tests/test_files/'
pdb_folder = resource_folder + 'pdb/'
membrane_folder = pdb_folder + 'membrane/'
ion_channels_folder = pdb_folder + 'ion_channels/'
metabolites_folder = pdb_folder + 'metabolites/'
transporters_folder = pdb_folder + 'transporters/'

In [102]:
''' Dimensions are in nanometers '''
location_areas = dict()
location_areas[0] = [0.45, Vector2(-20, -100)] # Cytosol of the neuron
location_areas[1] = [0.25, Vector2(20, 100)] # Cytosol of the astrocyte
location_areas[2] = [0.20, Vector2(-20, 20)] # Extracellular space
location_areas[3] = [0.0055, Vector2(0, 0)] # Capillaries (Not used)
location_areas[4] = [0.0, Vector2(0, 0)] # Synaptic part of the extracellular space (Not used)
location_areas[5] = [0.0575 * 0.45, Vector2(-100, -scene_size.x)] # Mitochondria of the neuron
location_areas[6] = [0.0575 * 0.25, Vector2(100, scene_size.x)] # Mitochondria of the astrocyte
location_areas[7] = [0.0, Vector2(0, 0)] # Vasculature (Not used)

## Neuron

In [188]:
ce.add_sphere([0, 10010,0], 10000,[1,1,0,0.5], 'NeuronSphere')

{'error': 0, 'id': 68, 'message': ''}

In [185]:
ce.add_sphere([0, 10110,0], 10000,[1,1,1,0.5], 'NeuronMitochodrionSphere')

{'error': 0, 'id': 65, 'message': ''}

In [187]:
ce.add_sphere([0, -10010,0], 10000,[0,0,1,0.5], 'AstrocyteSphere')

{'error': 0, 'id': 67, 'message': ''}

In [186]:
ce.add_sphere([0, -10110,0], 10000,[1,1,1,0.5], 'AstrocyteMitochodrionSphere')

{'error': 0, 'id': 66, 'message': ''}

In [31]:
def add_neuron(frame):
    name = 'Neuron'

    # Transporter
    transporter = Protein(
        sources=[transporters_folder + '4zwc.pdb'],
        occurences=20, position=Vector3(0.0, 0.0, 0.0),
        rotation=Quaternion(0.707, 0.707, 0.0, 0.0),
        load_non_polymer_chemicals=True, load_hydrogen=True, load_bonds=True
    )

    # Membrane definition
    membrane = ParametricMembrane(
        sources=[membrane_folder + 'new_mem.pdb'],
        occurences=membrane_nb_lipids,
        load_non_polymer_chemicals=True, load_bonds=True
    )

    # Cell definition
    neuron = Cell(
        name=name,
        size=membrane_size,
        shape=be.ASSEMBLY_SHAPE_SINUSOIDAL,
        extra_parameters=[membrane_height],
        membrane=membrane,
        random_position_seed=frame + 1, random_position_strength=0.025,
        random_rotation_seed=frame + 2, random_rotation_strength=0.05,
        receptor=transporter)

    # Add cell to scene
    status = be.add_cell(
        cell=neuron, representation=representation,
        position=Vector3(0, location_areas[0][1].x, 0), random_seed=1)


In [32]:
def add_neuron_mitochondrion(frame):
    name = 'NeuronMitochondrion'

    # Transporter
    transporter = Protein(
        sources=[transporters_folder + '4zwc.pdb'],
        occurences=0, position=Vector3(0.0, 0.0, 0.0),
        rotation=Quaternion(0.707, 0.707, 0.0, 0.0),
        load_non_polymer_chemicals=True, load_hydrogen=True, load_bonds=True
    )

    # Membrane definition
    membrane = ParametricMembrane(
        sources=[membrane_folder + 'new_mem.pdb'],
        occurences=membrane_nb_lipids,
        load_non_polymer_chemicals=True, load_bonds=True
    )

    # Cell definition
    neuron_mitochodrion = Cell(
        name=name,
        size=membrane_size,
        shape=be.ASSEMBLY_SHAPE_SINUSOIDAL,
        extra_parameters=[membrane_height],
        membrane=membrane,
        random_position_seed=frame + 3, random_position_strength=0.025,
        random_rotation_seed=frame + 4, random_rotation_strength=0.05,
        receptor=transporter)

    # Add cell to scene
    status = be.add_cell(
        cell=neuron_mitochodrion, representation=representation,
        position=Vector3(0, location_areas[5][1].x, 0), random_seed=2)

## Astrocyte

In [33]:
def add_astrocyte(frame):
    name = 'Astrocyte'

    # Transporter
    transporter = Protein(
        sources=[transporters_folder + '4pyp.pdb'],
        occurences=20, position=Vector3(0.0, 0.0, 0.0),
        rotation=Quaternion(0.707, 0.707, 0.0, 0.0),
        load_non_polymer_chemicals=True, load_hydrogen=True, load_bonds=True
    )

    # Membrane definition
    membrane = ParametricMembrane(
        sources=[membrane_folder + 'new_mem.pdb'],
        occurences=membrane_nb_lipids,
        load_non_polymer_chemicals=True, load_bonds=True)

    # Cell definition
    astrocyte = Cell(
        name=name,
        size=membrane_size,
        shape=be.ASSEMBLY_SHAPE_SINUSOIDAL,
        extra_parameters=[membrane_height],
        membrane=membrane,
        random_position_seed=frame + 5, random_position_strength=0.025,
        random_rotation_seed=frame + 6, random_rotation_strength=0.05,
        receptor=transporter)

    # Add cell to scene
    status = be.add_cell(
        cell=astrocyte, representation=representation,
        rotation=Quaternion(0.0, 1.0, 0.0, 0.0),
        position=Vector3(0, location_areas[1][1].x, 0),
        random_seed=3)

In [34]:
def add_astrocyte_mitochondrion(frame):
    name = 'AstrocyteMitochondrion'

    # Transporter
    transporter = Protein(
        sources=[transporters_folder + '4pyp.pdb'],
        occurences=0, position=Vector3(0.0, 0.0, 0.0),
        rotation=Quaternion(0.707, 0.707, 0.0, 0.0),
        load_non_polymer_chemicals=True, load_hydrogen=True, load_bonds=True
    )

    # Membrane definition
    membrane = ParametricMembrane(
        sources=[membrane_folder + 'new_mem.pdb'],
        occurences=membrane_nb_lipids,
        load_non_polymer_chemicals=True, load_bonds=True
    )

    # Cell definition
    astrocyte_mitochodrion = Cell(
        name=name,
        size=membrane_size,
        shape=be.ASSEMBLY_SHAPE_SINUSOIDAL,
        extra_parameters=[membrane_height],
        membrane=membrane,
        random_position_seed=frame + 7, random_position_strength=0.025,
        random_rotation_seed=frame + 8, random_rotation_strength=0.05,
        receptor=transporter)

    # Add cell to scene
    status = be.add_cell(
        cell=astrocyte_mitochodrion, representation=representation,
        rotation=Quaternion(0.0, 1.0, 0.0, 0.0),
        position=Vector3(0, location_areas[6][1].x, 0),
        random_seed=4)

## Materials

In [191]:
COLOR_MAPS = [
    'Accent', 'Accent_r', 'Blues', 'Blues_r', 'BrBG', 'BrBG_r', 'BuGn', 'BuGn_r',
    'BuPu', 'BuPu_r', 'CMRmap', 'CMRmap_r', 'Dark2', 'Dark2_r', 'GnBu', 'GnBu_r',
    'Greens', 'Greens_r', 'Greys', 'Greys_r', 'OrRd', 'OrRd_r', 'Oranges', 'Oranges_r',
    'PRGn', 'PRGn_r', 'Paired', 'Paired_r', 'Pastel1', 'Pastel1_r', 'Pastel2', 'Pastel2_r',
    'PiYG', 'PiYG_r', 'PuBu', 'PuBuGn', 'PuBuGn_r', 'PuBu_r', 'PuOr', 'PuOr_r',
    'PuRd', 'PuRd_r', 'Purples', 'Purples_r', 'RdBu', 'RdBu_r', 'RdGy', 'RdGy_r',
    'RdPu', 'RdPu_r', 'RdYlBu', 'RdYlBu_r', 'RdYlGn', 'RdYlGn_r', 'Reds', 'Reds_r',
    'Set1', 'Set1_r', 'Set2', 'Set2_r', 'Set3', 'Set3_r',
    'Wistia', 'Wistia_r', 'YlGn', 'YlGnBu', 'YlGnBu_r', 'YlGn_r', 'YlOrBr',
    'YlOrBr_r', 'YlOrRd', 'YlOrRd_r', 'afmhot', 'afmhot_r', 'autumn', 'autumn_r',
    'binary', 'binary_r', 'bone', 'bone_r', 'brg', 'brg_r', 'bwr', 'bwr_r', 'cool', 'cool_r',
    'coolwarm', 'coolwarm_r', 'copper', 'copper_r', 'cubehelix', 'cubehelix_r', 'flag',
    'flag_r', 'gist_earth', 'gist_earth_r', 'gist_gray', 'gist_gray_r', 'gist_heat',
    'gist_heat_r', 'gist_ncar', 'gist_ncar_r', 'gist_rainbow', 'gist_rainbow_r', 'gist_stern',
    'gist_stern_r', 'gist_yarg', 'gist_yarg_r', 'gnuplot', 'gnuplot2', 'gnuplot2_r',
    'gnuplot_r', 'gray', 'gray_r', 'hot', 'hot_r', 'hsv', 'hsv_r', 'icefire', 'icefire_r',
    'inferno', 'inferno_r', 'jet_r', 'magma', 'magma_r', 'mako', 'mako_r',
    'nipy_spectral', 'nipy_spectral_r', 'ocean', 'ocean_r', 'pink', 'pink_r', 'plasma',
    'plasma_r', 'prism', 'prism_r', 'rainbow', 'rainbow_r', 'rocket', 'rocket_r', 'seismic',
    'seismic_r', 'spring', 'spring_r', 'summer', 'summer_r',
    'tab10', 'tab10_r', 'tab20', 'tab20_r', 'tab20b', 'tab20b_r', 'tab20c', 'tab20c_r',
    'terrain', 'terrain_r', 'viridis', 'viridis_r', 'vlag', 'vlag_r', 'winter', 'winter_r'
]

def set_color_scheme(
        shading_mode, user_parameter=1.0, specular_exponent=5.0, glossiness=1.0):
    """
    Apply a default color scheme to all components in the scene

    :shading_mode: Shading mode (None, basic, diffuse, electron, etc)
    :user_parameter: User parameter specific to each shading mode
    :specular_exponent: Specular exponent for diffuse shading modes
    :glossiness: Glossiness
    """

    import seaborn as sns
    model_ids = be.get_model_ids()
    global_palette = sns.color_palette('rainbow', len(model_ids["ids"]))

    index = 0
    for model_id in model_ids["ids"]:
        model_name = be.get_model_name(model_id)['name']
        material_ids = be.get_material_ids(model_id)["ids"]
        nb_materials = len(material_ids)

        if model_name.find('Neuron') != -1 and model_name.find('Mitochondrion') == -1:
            palette = sns.color_palette("Blues", nb_materials)
            be.set_materials_from_palette(
                model_ids=[model_id],
                material_ids=material_ids,
                palette=palette,
                shading_mode=shading_mode,
                user_parameter=user_parameter,
                glossiness=glossiness,
                specular_exponent=specular_exponent,
            )
        if model_name.find('Astrocyte') != -1 and model_name.find('Mitochondrion') == -1:
            palette = sns.color_palette("Wistia", nb_materials)
            be.set_materials_from_palette(
                model_ids=[model_id],
                material_ids=material_ids,
                palette=palette,
                shading_mode=shading_mode,
                user_parameter=user_parameter,
                glossiness=glossiness,
                specular_exponent=specular_exponent,
            )
        if model_name.find('Mitochondrion') != -1:
            palette = sns.color_palette("Purples_r", nb_materials)
            be.set_materials_from_palette(
                model_ids=[model_id],
                material_ids=material_ids,
                palette=palette,
                shading_mode=shading_mode,
                user_parameter=user_parameter,
                glossiness=glossiness,
                specular_exponent=specular_exponent,
            )
        else:
            colors = list()
            shading_modes = list()
            user_parameters = list()
            glossinesses = list()
            specular_exponents = list()

            for m in material_ids:
                colors.append(global_palette[index])
                shading_modes.append(shading_mode)
                user_parameters.append(user_parameter)
                glossinesses.append(glossiness)
                specular_exponents.append(specular_exponent)

            be.set_materials(
                model_ids=[model_id],
                material_ids=material_ids,
                diffuse_colors=colors,
                specular_colors=colors,
                shading_modes=shading_modes,
                user_parameters=user_parameters,
                glossinesses=glossinesses,
                specular_exponents=specular_exponents
            )

        if model_name == 'NeuronSphere':
            print(model_name)
            palette = list()
            for i in range(nb_materials):
                palette.append([1,1,0])
            ce.set_material_extra_attributes(model_id)
            be.set_materials_from_palette(
                model_ids=[model_id],
                material_ids=material_ids,
                palette=palette,
                shading_mode=shading_mode,
                user_parameter=user_parameter,
                glossiness=glossiness,
                specular_exponent=specular_exponent,
                opacity=0.25
            )
        elif model_name == 'AstrocyteSphere':
            print(model_name)
            palette = list()
            for i in range(nb_materials):
                palette.append([0.0, 0.75, 1.0])
            ce.set_material_extra_attributes(model_id)
            be.set_materials_from_palette(
                model_ids=[model_id],
                material_ids=material_ids,
                palette=palette,
                shading_mode=shading_mode,
                user_parameter=user_parameter,
                glossiness=glossiness,
                specular_exponent=specular_exponent,
                opacity=0.25
            )
        elif model_name == 'AstrocyteMitochodrionSphere' or model_name == 'NeuronMitochodrionSphere':
            print(model_name)
            palette = list()
            for i in range(nb_materials):
                palette.append([153 / 255, 50 / 255, 204 / 255])
            ce.set_material_extra_attributes(model_id)
            be.set_materials_from_palette(
                model_ids=[model_id],
                material_ids=material_ids,
                palette=palette,
                shading_mode=shading_mode,
                user_parameter=user_parameter,
                glossiness=glossiness,
                specular_exponent=specular_exponent,
                opacity = 1.0
            )
        index += 1


set_color_scheme(shading_mode=be.SHADING_MODE_NONE, user_parameter=0.25, specular_exponent=100)
be.core_api().set_renderer()

NeuronMitochodrionSphere
AstrocyteMitochodrionSphere


True

In [36]:
def setup_frame(frame):
    be.reset()
    be.set_general_settings(model_visibility_on_creation=False)
    print('Loading neuron membrane...')
    add_neuron(frame)
    print('Loading neuron mitochondrion membrane...')
    add_neuron_mitochondrion(frame)
    print('Loading astrocyte membrane...')
    add_astrocyte(frame)
    print('Loading astrocyte mitochondrion membrane...')
    add_astrocyte_mitochondrion(frame)
    print('Applying materials...')
    set_color_scheme(shading_mode=be.SHADING_MODE_DIFFUSE, specular_exponent=50)
    print('Building geometry...')
    be.set_models_visibility(True)
    print('Done')

In [37]:
be.set_rendering_quality(be.RENDERING_QUALITY_HIGH)

params = be.core_api().BioExplorerRendererParams()
params.shadows = 1.0
params.soft_shadows = 0.5
params.use_hardware_randomizer = True
params.fog_start = 250.0
params.fog_thickness = 250.0
params.gi_distance = 20.0
params.gi_weight = 0.2
params.gi_samples = 3
params = be.core_api().set_renderer_params(params)
params = be.core_api().set_renderer(head_light=True)

## Snapshots

In [38]:
from bioexplorer import MovieMaker
mm = MovieMaker(be)

In [183]:
be.core_api().set_camera(
    orientation = [-0.707, -0.707, 0.0, 0.0],
    position = [0.7841586723976661, 3.6684584451627895, -360.03218098964624],
    target = [0.7841586723976661, 3.6684584451626847, -25.817442464876535]
)
# setup_frame(0)

True

In [17]:
from bioexplorer import Widgets
w = Widgets(be)
w.display_focal_distance()

In [18]:
if generate_snapshot:
    be.core_api().set_camera(current='orthographic')
    params = be.core_api().OrthographicCameraParams()
    params.height = 100.0
    be.core_api().set_camera_params(params)

    be.core_api().set_camera(
        orientation = [-0.707, -0.707, 0.0, 0.0],
        position = [0.7841586723976661, 3.6684584451627895, -360.03218098964624],
        target = [0.7841586723976661, 3.6684584451626847, -25.817442464876535]
    )

    mm.create_snapshot(
        base_name = 'Metabolism_Molecular_Level_4k_v10',
        path='/gpfs/bbp.cscs.ch/project/proj129/images/metabolism/4K',
        size=[2160, 3840], samples_per_pixel=64
)

## Simulation movie

In [19]:
mm.get_camera()

{'apertureRadius': 0.0,
 'direction': [0.0, 0.0, 0.9993959999999997],
 'focusDistance': 1000000.0,
 'origin': [0.7841586723976661, 3.6684584451627895, -360.03218098964624],
 'up': [0.9996979999999999, 0.0003020000000001355, -0.0]}

In [152]:
nb_frames = 100
key_frames = [
    { # 0
        'apertureRadius': 0.02,
        'direction': [0.0, 0.0, 1.0],
        'focusDistance': 260.0,
        'origin': [0.7841586723976661, 3.6684584451627895, -360.03218098964624],
        'up': [0.9996979999999999, 0.0003020000000001355, 0.0]
    },
    { # 100
        'apertureRadius': 0.02,
        'direction': [0.0, 0.0, 1.0],
        'focusDistance': 260.0,
        'origin': [0.7841586723976661, 3.6684584451627895, -160.03218098964624],
        'up': [0.9996979999999999, 0.0003020000000001355, 0.0]
    },
    { # 200
        'apertureRadius': 0.02,
        'direction': [-5.551115123125783e-17, -1.0, 0.0],
        'focusDistance': 1000000.0,
        'origin': [28.167633056640625, 15.00307907686704, -13.82595157623291],
        'up': [0.995946358535887, 0.0, -0.08994915741187692]
    },
    { # 300
        'apertureRadius': 0.02,
        'direction': [0.0, -1.0, 0.0],
        'focusDistance': 1000000.0,
        'origin': [30.811624840998768, -14.626182907516387, -13.686327604222097],
        'up': [0.9958837897803697, -0.005724790808323554, -0.0904582999115569]
    },
    { # 400
        'apertureRadius': 0.02,
        'direction': [0.0,  -1.0,  0.0],
        'focusDistance': 1000000.0,
        'origin': [29.476409912109375, -41.00079776535672, -13.756837844848633],
        'up': [1.2246467991473535e-16, -4.440892098500626e-16, -1.0000000000000002]
    },
    { # 400
        'apertureRadius': 0.02,
        'direction': [0.0,  -1.0,  0.0],
        'focusDistance': 1000000.0,
        'origin': [29.476409912109375, -41.00079776535672, -13.756837844848633],
        'up': [1.2246467991473535e-16, -4.440892098500626e-16, -1.0000000000000002]
    }
]

mm.build_camera_path(key_frames, nb_frames, nb_frames / 4)
# mm.build_camera_path(key_frames, nb_frames, 1)
print(mm.get_nb_frames())

501


In [134]:
be.core_api().set_camera(current='bio_explorer_perspective')
frame = 0
mm.set_current_frame(frame)
key_frame = mm.get_key_frame(frame)
print(key_frame)

params = be.core_api().BioExplorerPerspectiveCameraParams()
params.aperture_radius = 0 #key_frame[3]
params.focus_distance = key_frame[4]
be.core_api().set_camera_params(params)

be.core_api().set_renderer()

[(0.7841586723976661, 3.6684584451627886, -354.75218098964615), (0.0, 0.0, 1.0), (0.9996979999999996, 0.0003020000000001355, 0.0), 0.020000000000000004, 260.0]


True

In [192]:
import time
from IPython.display import clear_output

draft = False

output_folder = '/gpfs/bbp.cscs.ch/project/proj129/movies/metabolism/4K/mitochondria'
for frame in range(mm.get_nb_frames()):
    clear_output()
    print('Frame %d' % frame)
    mm.set_current_frame(frame)

    if draft:
        time.sleep(0.1)
    else:
        mm.create_snapshot(
            size=[3840, 2160], samples_per_pixel=8,
            path=output_folder, base_name='%05d' % frame)


Frame 500


IntProgress(value=0, description='In progress...')